<a href="https://colab.research.google.com/github/jsung0228/code/blob/main/Combining_Multiple_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#In this tutorial we will practice how to combine datasets and discover if we can find more relationships

import pandas as pd
import numpy as np


unemp_county = pd.read_csv("https://raw.githubusercontent.com/jsung0228/code/main/DataSets/US_unemployment_rate.csv")
unemp_county.head()


,Year,Month,State,County,Rate
0,1999,June,Texas,Ward County,13.3
1,1999,June,Texas,Baylor County,4.0
2,1999,June,Texas,Scurry County,10.2
3,1999,June,Texas,Collingsworth County,1.3
4,1999,June,Texas,Cherokee County,4.6


In [8]:
#Let's bringin the datasets for minimum wage tables

df = pd.read_csv('https://raw.githubusercontent.com/jsung0228/code/main/DataSets/Minimum%20Wage%20Data.csv', encoding="latin")

#First let's make a minimum wage table for each year for each states
act_min_wage = pd.DataFrame()
for name, group in df.groupby("State"):
    if act_min_wage.empty:
        act_min_wage = group.set_index("Year")[["Department.Of.Labor.Cleaned.Low.Value.2020.Dollars"]].rename(columns={"Department.Of.Labor.Cleaned.Low.Value.2020.Dollars":name})
    else:
        act_min_wage = act_min_wage.join(group.set_index("Year")[["Department.Of.Labor.Cleaned.Low.Value.2020.Dollars"]].rename(columns={"Department.Of.Labor.Cleaned.Low.Value.2020.Dollars":name}))

act_min_wage = act_min_wage.replace(0,np.NaN).dropna(axis=1)
act_min_wage.head()

,Alaska,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Guam,Hawaii,Idaho,Indiana,Kentucky,Maine,Maryland,Massachusetts,Michigan,Minnesota,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Dakota,Utah,Vermont,Washington,West Virginia,Wisconsin,Wyoming
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1968,15.61,1.16,12.26,7.43,10.41,9.29,9.29,9.29,9.29,8.55,8.55,4.83,10.41,7.43,11.89,9.29,5.20,7.43,9.29,10.41,10.41,8.55,11.89,7.43,7.43,5.57,7.43,9.29,8.55,10.41,3.16,7.43,10.41,11.89,7.43,9.29,8.92
1969,14.80,1.10,11.63,7.05,9.87,8.81,8.81,8.81,8.81,8.11,8.11,4.58,9.87,7.05,11.28,8.81,4.93,7.05,8.81,9.87,9.87,8.11,11.28,7.05,7.05,5.29,7.05,8.81,8.11,9.87,3.00,7.05,9.87,11.28,7.05,8.81,8.46
1970,14.00,7.33,11.00,6.67,10.67,8.33,10.67,10.67,10.67,8.33,8.33,4.33,10.67,8.67,10.67,8.33,4.67,6.67,8.67,9.67,10.00,8.67,10.67,8.33,6.67,5.00,6.67,8.33,8.67,10.67,6.67,6.67,10.67,10.67,6.67,8.67,8.67
1971,13.41,7.03,10.54,6.39,10.22,7.98,10.22,10.22,10.22,7.98,7.98,4.15,10.22,8.30,10.22,7.98,4.47,6.39,8.30,9.26,9.58,8.30,10.22,7.98,6.39,4.79,6.39,7.98,8.30,10.22,6.39,6.39,10.22,10.22,6.39,8.30,8.30
1972,12.99,7.43,10.21,6.19,11.45,9.90,9.90,11.76,9.90,8.66,7.74,4.02,8.66,9.90,10.83,9.90,4.64,6.19,9.90,9.90,9.28,8.04,11.45,8.97,6.19,4.64,8.66,7.74,9.90,9.90,6.19,7.43,9.90,9.90,7.43,8.97,9.28


We want to add a minimum wage data on the unemployment data

In [9]:
def get_min_wage(year, state):
  try: 
    return act_min_wage.loc[year][state]
  except:
    return np.NaN
get_min_wage(2012, "Colorado")

8.61

In [13]:
#So the way we will approach it, is by using the mapping function in python which will iterate a function on a given list

%%time
unemp_county['min_wage'] = list(map(get_min_wage, unemp_county['Year'], unemp_county['State']))

CPU times: user 1min 2s, sys: 190 ms, total: 1min 2s
Wall time: 1min 9s


In [15]:
unemp_county.tail()

,Year,Month,State,County,Rate,min_wage
569098,2016,November,Maine,Somerset County,5.4,8.08
569099,2016,November,Maine,Oxford County,4.6,8.08
569100,2016,November,Maine,Knox County,3.6,8.08
569101,2016,November,Maine,Piscataquis County,5.6,8.08
569102,2016,November,Maine,Aroostook County,5.7,8.08


In [16]:
unemp_county[['Rate','min_wage']].corr()

,Rate,min_wage
Rate,1.00000,0.22897
min_wage,0.22897,1.00000


In [17]:
unemp_county[['Rate','min_wage']].cov()

,Rate,min_wage
Rate,8.062240,1.024039
min_wage,1.024039,2.484621


In [18]:
pres16 = pd.read_csv("https://raw.githubusercontent.com/jsung0228/code/main/DataSets/pres16results.csv")
pres16.head()

,county,fips,cand,st,pct_report,votes,total_votes,pct,lead
0,NaN,US,Donald Trump,US,0.9951,60350241.0,127592176.0,0.472993,Donald Trump
1,NaN,US,Hillary Clinton,US,0.9951,60981118.0,127592176.0,0.477938,Donald Trump
2,NaN,US,Gary Johnson,US,0.9951,4164589.0,127592176.0,0.032640,Donald Trump
3,NaN,US,Jill Stein,US,0.9951,1255968.0,127592176.0,0.009844,Donald Trump
4,NaN,US,Evan McMullin,US,0.9951,451636.0,127592176.0,0.003540,Donald Trump


In [23]:
county_2015 = unemp_county[(unemp_county['Year']==2015)&(unemp_county['Month']=='February')]
county_2015.head()

,Year,Month,State,County,Rate,min_wage
501263,2015,February,Mississippi,Newton County,6.1,NaN
501264,2015,February,Mississippi,Panola County,9.4,NaN
501265,2015,February,Mississippi,Monroe County,7.9,NaN
501266,2015,February,Mississippi,Hinds County,6.1,NaN
501267,2015,February,Mississippi,Kemper County,10.6,NaN
